## Combined SOC Prediction - Model Evaluation
Uses discharge cycles from both the LG and Panasonic datasets.

LG 18650HG2 Li-ion Battery Data - https://data.mendeley.com/datasets/cp3473x7xv/3

Panasonic 18650PF Li-ion Battery Data - https://data.mendeley.com/datasets/wykht8y7tg/1

Kollmeyer, Philip; Vidal, Carlos; Naguib, Mina; Skells, Michael  (2020), “LG 18650HG2 Li-ion Battery Data and Example Deep Neural Network xEV SOC Estimator Script”, Mendeley Data, V3, doi: 10.17632/cp3473x7xv.3

Kollmeyer, Phillip (2018), “Panasonic 18650PF Li-ion Battery Data”, Mendeley Data, V1, doi: 10.17632/wykht8y7tg.1

In [1]:
import os
import sys
import IPython
import IPython.display
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import collections
from datetime import datetime, timedelta

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

base_path = '../../'

sys.path.append(base_path)
from training import utils

panasonic_data_path = base_path + 'datasets/Panasonic_18650PF'
lg_data_path = base_path + 'datasets/LG_18650HG2'

resample_1hz = True
vi_averages = True

2023-04-03 02:52:26.323542: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 02:52:27.337589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-03 02:52:27.337650: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-03 02:52:27.452516: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-03 02:52:29.749809: W tensorflow/stream_executor/platform/de

## Load LG, Merged Test Data (1Hz)

In [2]:
merged_dnn_cnn_columns = ['Voltage', 'Current', 'Voltage Average', 'Current Average', 'Temperature', 'Capacity']
lg_dnn_cnn_columns = ['Voltage', 'Current', 'Temperature', 'Power', 'Capacity', 'Voltage Average', 'Current Average', 'Power Average']
lstm_columns = ['Voltage', 'Current', 'Voltage Average', 'Current Average', 'Temperature', 'Capacity']

In [3]:
lg_charge_cycles = ['Charge']

lg_test_temps = ['n10degC', '0degC', '10degC', '25degC']
lg_test_discharge_cycles = ['UDDS', 'HWFET', 'Mixed1', 'Mixed2' ]

panasonic_test_temps = ['n10degC', '0degC', '10degC', '25degC']
panasonic_test_discharge_cycles = ['US06', 'LA92', 'Cycle_3', 'Cycle_4']

In [4]:
lg_test_files = utils.lg_get_files(lg_data_path, 
                                   lg_test_discharge_cycles,
                                   lg_charge_cycles, 
                                   lg_test_temps)

panasonic_test_files = utils.panasonic_get_files(panasonic_data_path, 
                                                 panasonic_test_discharge_cycles,
                                                 panasonic_test_temps)

for file in panasonic_test_files:
    if "UDDS_LA92" in file or "LA92_NN" in file:
        panasonic_test_files.remove(file)
        
lg_test, lg_test_norm = utils.lg_create_dataset(lg_test_files, 
                                                lg_test_discharge_cycles, 
                                                lg_charge_cycles, 
                                                vi_averages, 
                                                resample_1hz)

panasonic_test, panasonic_test_norm = utils.panasonic_create_dataset(panasonic_test_files, 
                                                                     panasonic_test_discharge_cycles, 
                                                                     vi_averages, 
                                                                     resample_1hz)

../../datasets/LG_18650HG2/25degC/551_UDDS.csv 159646
../../datasets/LG_18650HG2/25degC/551_HWFET.csv 6002
../../datasets/LG_18650HG2/25degC/551_Mixed1.csv 77227
../../datasets/LG_18650HG2/25degC/551_Mixed2.csv 79172
../../datasets/LG_18650HG2/10degC/567_Mixed1.csv 72329
../../datasets/LG_18650HG2/10degC/567_Mixed2.csv 77752
../../datasets/LG_18650HG2/10degC/576_UDDS.csv 148898
../../datasets/LG_18650HG2/10degC/576_HWFET.csv 52878
../../datasets/LG_18650HG2/0degC/589_UDDS.csv 140435
../../datasets/LG_18650HG2/0degC/589_HWFET.csv 49678
../../datasets/LG_18650HG2/0degC/589_Mixed1.csv 67312
../../datasets/LG_18650HG2/0degC/589_Mixed2.csv 72303
../../datasets/LG_18650HG2/n10degC/596_UDDS.csv 123602
../../datasets/LG_18650HG2/n10degC/596_HWFET.csv 44903
../../datasets/LG_18650HG2/n10degC/601_Mixed1.csv 62622
../../datasets/LG_18650HG2/n10degC/601_Mixed2.csv 65254
../../datasets/Panasonic_18650PF/0degC/Drive cycles/06-01-17_10.36 0degC_LA92_Pan18650PF.csv 82666
../../datasets/Panasonic_18650

In [5]:
lg_test_norm.describe()

Time Stamp,Voltage,Current,Temperature,Power,Capacity,Voltage Average,Current Average,Power Average
count,112931.000000,112931.000000,112931.000000,112931.000000,112931.000000,112931.000000,112931.000000,112931.000000
mean,0.633440,0.693250,0.463483,0.619687,0.508268,0.610681,0.764340,0.746015
std,0.183403,0.091121,0.334283,0.097508,0.278759,0.209849,0.206260,0.225943
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.510840,0.646543,0.268222,0.566116,0.278374,0.443814,0.658521,0.644669
50%,0.635422,0.707464,0.533528,0.632830,0.491650,0.603989,0.824125,0.808537
75%,0.776015,0.735598,0.935860,0.663817,0.753426,0.788472,0.932516,0.931963
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
panasonic_test_norm.describe()

,Voltage,Current,Temperature,Power,Capacity,Voltage Average,Current Average,Power Average
count,118259.000000,118259.000000,118259.000000,118259.000000,118259.000000,118259.000000,118259.000000,118259.000000
mean,0.642341,0.636101,0.524135,0.555323,0.482904,0.555842,0.665536,0.620162
std,0.150859,0.067816,0.281902,0.078962,0.289502,0.215397,0.216903,0.230134
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.542082,0.606577,0.283354,0.518430,0.234748,0.399273,0.563470,0.514581
50%,0.644041,0.655559,0.511664,0.576259,0.497003,0.557021,0.742143,0.699572
75%,0.754314,0.670344,0.845029,0.594524,0.740011,0.720476,0.815108,0.778447
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
merged_test_norm = pd.concat([lg_test_norm, panasonic_test_norm], ignore_index=True)

In [8]:
merged_test_norm.describe()

,Voltage,Current,Temperature,Power,Capacity,Voltage Average,Current Average,Power Average
count,231190.000000,231190.000000,231190.000000,231190.000000,231190.000000,231190.000000,231190.000000,231190.000000
mean,0.637993,0.664017,0.494508,0.586764,0.495294,0.582629,0.713800,0.681639
std,0.167606,0.084996,0.310087,0.094174,0.284587,0.214463,0.217454,0.236612
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.528539,0.622617,0.271137,0.538960,0.257618,0.426285,0.597594,0.565364
50%,0.640478,0.668205,0.512491,0.591825,0.494320,0.580401,0.774217,0.736153
75%,0.763254,0.716201,0.845029,0.643955,0.744572,0.754441,0.888894,0.865627
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Evaluate LG Test data against LG and Combined models

In [9]:
model_eval = {}

In [10]:
# Switch to LG models
cell_type = "lg"

In [11]:
## LOAD LG DNN MODEL
model_type = "dnn_model"
pre_trained_path = base_path + 'pre-trained/' + cell_type + '_' + model_type
lg_dnn_model = tf.keras.models.load_model(pre_trained_path)

2023-04-03 02:53:03.345047: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-03 02:53:03.345107: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-03 02:53:03.345136: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (AP-Surface): /proc/driver/nvidia/version does not exist
2023-04-03 02:53:03.345509: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
## LOAD LG CNN MODEL
model_type = "cnn_model"
pre_trained_path = base_path + 'pre-trained/' + cell_type + '_' + model_type
lg_cnn_model = tf.keras.models.load_model(pre_trained_path)

In [13]:
# LG test data
lg_test_features = lg_test_norm[lg_dnn_cnn_columns].copy()
lg_test_labels = lg_test_features.pop('Capacity')

# Combined test data
merged_test_features = merged_test_norm[lg_dnn_cnn_columns].copy()
merged_test_labels = merged_test_features.pop('Capacity')

In [14]:
# Evaluate LG test against LG DNN
model_eval['lg_dnn_lg_data'] = lg_dnn_model.evaluate(lg_test_features, lg_test_labels, verbose=2)
# Evaluate LG test against LG CNN
model_eval['lg_cnn_lg_data'] = lg_cnn_model.evaluate(lg_test_features, lg_test_labels, verbose=2)
# Evaluate Combined test against LG DNN
model_eval['lg_dnn_merged_data'] = lg_dnn_model.evaluate(merged_test_features, merged_test_labels, verbose=2)
# Evaluate Combined test against LG CNN
model_eval['lg_cnn_merged_data'] = lg_cnn_model.evaluate(merged_test_features, merged_test_labels, verbose=2)

3530/3530 - 5s - loss: 0.0193 - mse: 6.5232e-04 - mae: 0.0193 - mape: 306.0367 - rmse: 0.0255 - 5s/epoch - 2ms/step
3530/3530 - 6s - loss: 0.0213 - 6s/epoch - 2ms/step
7225/7225 - 11s - loss: 0.0308 - mse: 0.0018 - mae: 0.0308 - mape: 1241606.7500 - rmse: 0.0425 - 11s/epoch - 2ms/step
7225/7225 - 11s - loss: 0.0325 - 11s/epoch - 2ms/step


In [15]:
# Switch to combined models
cell_type = "comb"

In [16]:
## LOAD COMBINED DNN MODEL
model_type = "dnn_model"
pre_trained_path = base_path + 'pre-trained/' + cell_type + '_' + model_type
combined_dnn_model = tf.keras.models.load_model(pre_trained_path)

In [17]:
## LOAD COMBINED CNN MODEL
model_type = "cnn_model"
pre_trained_path = base_path + 'pre-trained/' + cell_type + '_' + model_type
combined_cnn_model = tf.keras.models.load_model(pre_trained_path)

In [18]:
# LG test data
lg_test_features = lg_test_norm[merged_dnn_cnn_columns].copy()
lg_test_labels = lg_test_features.pop('Capacity')

# Combined test data
merged_test_features = merged_test_norm[merged_dnn_cnn_columns].copy()
merged_test_labels = merged_test_features.pop('Capacity')

In [19]:
# Evaluate LG test against LG DNN
model_eval['combined_dnn_lg_data'] = combined_dnn_model.evaluate(lg_test_features, lg_test_labels, verbose=2)
# Evaluate LG test against LG CNN
model_eval['combined_cnn_lg_data'] = combined_cnn_model.evaluate(lg_test_features, lg_test_labels, verbose=2)
# Evaluate Combined test against LG DNN
model_eval['combined_dnn_merged_data'] = combined_dnn_model.evaluate(merged_test_features, merged_test_labels, verbose=2)
# Evaluate Combined test against LG CNN
model_eval['combined_cnn_merged_data'] = combined_cnn_model.evaluate(merged_test_features, merged_test_labels, verbose=2)

3530/3530 - 6s - loss: 0.0136 - mse: 2.9189e-04 - mae: 0.0136 - mape: 15.8433 - rmse: 0.0171 - 6s/epoch - 2ms/step
3530/3530 - 6s - loss: 0.0110 - 6s/epoch - 2ms/step
7225/7225 - 13s - loss: 0.0272 - mse: 0.0015 - mae: 0.0272 - mape: 548015.1250 - rmse: 0.0387 - 13s/epoch - 2ms/step
7225/7225 - 12s - loss: 0.0245 - 12s/epoch - 2ms/step


## Load LG, Merged Test Data (Raw)

In [20]:
# Raw
resample_1hz = False
vi_averages = True

# Switch to LG models
cell_type = "lg"

In [21]:
lg_test_files = utils.lg_get_files(lg_data_path, 
                                   lg_test_discharge_cycles,
                                   lg_charge_cycles, 
                                   lg_test_temps)

panasonic_test_files = utils.panasonic_get_files(panasonic_data_path, 
                                                 panasonic_test_discharge_cycles,
                                                 panasonic_test_temps)

for file in panasonic_test_files:
    if "UDDS_LA92" in file or "LA92_NN" in file:
        panasonic_test_files.remove(file)
        
lg_test, lg_test_norm = utils.lg_create_dataset(lg_test_files, 
                                                lg_test_discharge_cycles, 
                                                lg_charge_cycles, 
                                                vi_averages, 
                                                resample_1hz)

panasonic_test, panasonic_test_norm = utils.panasonic_create_dataset(panasonic_test_files, 
                                                                     panasonic_test_discharge_cycles, 
                                                                     vi_averages, 
                                                                     resample_1hz)

# create lstm dataset
steps = 300

lg_test_x, lg_test_y = utils.create_lstm_dataset(lg_test_norm,steps)
lg_test_y = utils.keep_only_y_end(lg_test_y, steps)

../../datasets/LG_18650HG2/25degC/551_UDDS.csv 159646
../../datasets/LG_18650HG2/25degC/551_HWFET.csv 6002
../../datasets/LG_18650HG2/25degC/551_Mixed1.csv 77227
../../datasets/LG_18650HG2/25degC/551_Mixed2.csv 79172
../../datasets/LG_18650HG2/10degC/567_Mixed1.csv 72329
../../datasets/LG_18650HG2/10degC/567_Mixed2.csv 77752
../../datasets/LG_18650HG2/10degC/576_UDDS.csv 148898
../../datasets/LG_18650HG2/10degC/576_HWFET.csv 52878
../../datasets/LG_18650HG2/0degC/589_UDDS.csv 140435
../../datasets/LG_18650HG2/0degC/589_HWFET.csv 49678
../../datasets/LG_18650HG2/0degC/589_Mixed1.csv 67312
../../datasets/LG_18650HG2/0degC/589_Mixed2.csv 72303
../../datasets/LG_18650HG2/n10degC/596_UDDS.csv 123602
../../datasets/LG_18650HG2/n10degC/596_HWFET.csv 44903
../../datasets/LG_18650HG2/n10degC/601_Mixed1.csv 62622
../../datasets/LG_18650HG2/n10degC/601_Mixed2.csv 65254
../../datasets/Panasonic_18650PF/0degC/Drive cycles/06-01-17_10.36 0degC_LA92_Pan18650PF.csv 82666
../../datasets/Panasonic_18650

In [22]:
lg_test_files = utils.lg_get_files(lg_data_path, 
                                   lg_test_discharge_cycles,
                                   lg_charge_cycles, 
                                   lg_test_temps)

panasonic_test_files = utils.panasonic_get_files(panasonic_data_path, 
                                                 panasonic_test_discharge_cycles,
                                                 panasonic_test_temps)

for file in panasonic_test_files:
    if "UDDS_LA92" in file or "LA92_NN" in file:
        panasonic_test_files.remove(file)
        
lg_test, lg_test_norm = utils.lg_create_dataset(lg_test_files, 
                                                lg_test_discharge_cycles, 
                                                lg_charge_cycles, 
                                                vi_averages, 
                                                resample_1hz)

panasonic_test, panasonic_test_norm = utils.panasonic_create_dataset(panasonic_test_files, 
                                                                     panasonic_test_discharge_cycles, 
                                                                     vi_averages, 
                                                                     resample_1hz)

merged_test_norm = pd.concat([lg_test_norm, panasonic_test_norm], ignore_index=True)

# create lstm dataset
steps = 300

merged_test_x, merged_test_y = utils.create_lstm_dataset(merged_test_norm,steps)
merged_test_y = utils.keep_only_y_end(merged_test_y, steps)

../../datasets/LG_18650HG2/25degC/551_UDDS.csv 159646
../../datasets/LG_18650HG2/25degC/551_HWFET.csv 6002
../../datasets/LG_18650HG2/25degC/551_Mixed1.csv 77227
../../datasets/LG_18650HG2/25degC/551_Mixed2.csv 79172
../../datasets/LG_18650HG2/10degC/567_Mixed1.csv 72329
../../datasets/LG_18650HG2/10degC/567_Mixed2.csv 77752
../../datasets/LG_18650HG2/10degC/576_UDDS.csv 148898
../../datasets/LG_18650HG2/10degC/576_HWFET.csv 52878
../../datasets/LG_18650HG2/0degC/589_UDDS.csv 140435
../../datasets/LG_18650HG2/0degC/589_HWFET.csv 49678
../../datasets/LG_18650HG2/0degC/589_Mixed1.csv 67312
../../datasets/LG_18650HG2/0degC/589_Mixed2.csv 72303
../../datasets/LG_18650HG2/n10degC/596_UDDS.csv 123602
../../datasets/LG_18650HG2/n10degC/596_HWFET.csv 44903
../../datasets/LG_18650HG2/n10degC/601_Mixed1.csv 62622
../../datasets/LG_18650HG2/n10degC/601_Mixed2.csv 65254
../../datasets/Panasonic_18650PF/0degC/Drive cycles/06-01-17_10.36 0degC_LA92_Pan18650PF.csv 82666
../../datasets/Panasonic_18650

In [23]:
## LOAD LG LSTM MODEL
model_type = "lstm_model"
pre_trained_path = base_path + 'pre-trained/' + cell_type + '_' + model_type
lg_lstm_model = tf.keras.models.load_model(pre_trained_path)

In [24]:
# Evaluate LG test against LG DNN
model_eval['lg_lstm_lg_data'] = lg_lstm_model.evaluate(lg_test_x, lg_test_y, verbose=2)
# Evaluate LG test against LG CNN
model_eval['lg_lstm_combined_data'] = lg_lstm_model.evaluate(merged_test_x, merged_test_y, verbose=2)

118/118 - 61s - loss: 0.0028 - mse: 0.0057 - mae: 0.0487 - mape: 999.4768 - rmse: 0.0755 - 61s/epoch - 519ms/step
241/241 - 137s - loss: 0.0030 - mse: 0.0061 - mae: 0.0508 - mape: 2463013.5000 - rmse: 0.0780 - 137s/epoch - 567ms/step


In [25]:
# Switch to combined LSTM model
cell_type = "comb"

In [26]:
## LOAD COMBINED LSTM MODEL
model_type = "lstm_model"
pre_trained_path = base_path + 'pre-trained/' + cell_type + '_' + model_type
combined_lstm_model = tf.keras.models.load_model(pre_trained_path)

In [27]:
# Evaluate LG test against LG DNN
model_eval['combined_lstm_lg_data'] = combined_lstm_model.evaluate(lg_test_x, lg_test_y, verbose=2)
# Evaluate LG test against LG CNN
model_eval['combined_lstm_combined_data'] = combined_lstm_model.evaluate(merged_test_x, merged_test_y, verbose=2)

118/118 - 70s - loss: 0.0011 - mse: 0.0023 - mae: 0.0203 - mape: 228.5857 - rmse: 0.0476 - 70s/epoch - 594ms/step
241/241 - 145s - loss: 0.0014 - mse: 0.0028 - mae: 0.0298 - mape: 1500376.2500 - rmse: 0.0528 - 145s/epoch - 604ms/step


In [28]:
model_eval

{'lg_dnn_lg_data': [0.019250744953751564,
  0.0006523223710246384,
  0.019250744953751564,
  306.03668212890625,
  0.025540603324770927],
 'lg_cnn_lg_data': 0.0212621558457613,
 'lg_dnn_merged_data': [0.030848322436213493,
  0.001809990731999278,
  0.030848322436213493,
  1241606.75,
  0.042543984949588776],
 'lg_cnn_merged_data': 0.032491691410541534,
 'combined_dnn_lg_data': [0.013580014929175377,
  0.0002918858081102371,
  0.013580014929175377,
  15.843297958374023,
  0.017084665596485138],
 'combined_cnn_lg_data': 0.011027810163795948,
 'combined_dnn_merged_data': [0.027165768668055534,
  0.0014998017577454448,
  0.027165768668055534,
  548015.125,
  0.03872727230191231],
 'combined_cnn_merged_data': 0.024505220353603363,
 'lg_lstm_lg_data': [0.002848092932254076,
  0.005696185864508152,
  0.04869244247674942,
  999.476806640625,
  0.07547307759523392],
 'lg_lstm_combined_data': [0.003044678596779704,
  0.006089357193559408,
  0.050846148282289505,
  2463013.5,
  0.0780343338847160